# Predviđanje prsustva matice u košnici

In [17]:
import numpy as nm
import pandas as pd
from sklearn.model_selection import train_test_split
from math import pow
from sklearn import preprocessing
from numpy import savetxt
import seaborn as sns
import matplotlib.pyplot as plt


## Učitavanje podataka:

In [18]:
datasetStart = pd.read_csv("smart-bee-colony-monitor.csv")
datasetStart.head(5)

,device,hive number,date,hive temp,hive humidity,hive pressure,weather temp,weather humidity,weather pressure,wind speed,...,rain,lat,long,file name,queen presence,queen acceptance,frames,target,time,queen status
0,1,5,2022-06-08 14:52:28,36.42,30.29,1007.45,26.68,52,1013,8.75,...,0,37.29,-121.95,2022-06-08--14-52-28_1.raw,1,2,8,0,0.583,0
1,1,5,2022-06-08 15:51:41,33.56,33.98,1006.93,25.99,53,1012,10.29,...,0,37.29,-121.95,2022-06-08--15-51-41_1.raw,1,2,8,0,0.625,0
2,1,5,2022-06-08 17:21:53,29.01,42.73,1006.68,24.49,56,1012,8.75,...,0,37.29,-121.95,2022-06-08--17-21-53_1.raw,0,0,8,1,0.708,1
3,1,5,2022-06-08 18:20:59,30.51,36.74,1006.68,22.97,59,1012,8.23,...,0,37.29,-121.95,2022-06-08--18-20-59_1.raw,0,0,8,1,0.750,1
4,1,5,2022-06-08 19:20:04,30.32,35.55,1006.58,21.52,61,1012,7.20,...,0,37.29,-121.95,2022-06-08--19-20-04_1.raw,0,0,8,1,0.792,1


In [19]:
shape = datasetStart.shape
print('Velicina dataseta')
print(shape)

Velicina dataseta
(1275, 23)


# Priprema podataka i deskriptivna analiza 

In [16]:
print('Nedostajuće vrednosti po kolonama:')
print(datasetStart.isnull().sum())
print('Ukupan broj nedostajucih vrednosti: ', datasetStart.isnull().sum().sum())

print('\nTipovi podataka')
print(datasetStart.info())

print('\nSrednja vrednost izražena po kolonama:')
print(datasetStart.mean())

print('\nSrednja vrednost izražena po redovima:')
print(datasetStart.mean(axis=1))
 
print('\nMedijana:')
print(datasetStart.median())

print('\nMode:')
print(datasetStart.mode())

print('\nVarijansa:')
print(datasetStart.var())

print('\Duplirani podaci:')
print(datasetStart.duplicated().sum())

<>:23: SyntaxWarning: invalid escape sequence '\D'
<>:23: SyntaxWarning: invalid escape sequence '\D'
C:\Users\Maja\AppData\Local\Temp\ipykernel_2696\486812465.py:23: SyntaxWarning: invalid escape sequence '\D'
  print('\Duplirani podaci:')
C:\Users\Maja\AppData\Local\Temp\ipykernel_2696\486812465.py:23: SyntaxWarning: invalid escape sequence '\D'
  print('\Duplirani podaci:')


Nedostajuće vrednosti po kolonama:
device                0
hive number           0
date                  0
hive temp             0
hive humidity         0
hive pressure         0
weather temp          4
weather humidity      0
weather pressure      0
wind speed            4
gust speed          994
weatherID             0
cloud coverage        0
rain                  0
lat                   4
long                  4
file name             0
queen presence        0
queen acceptance      0
frames                0
target                0
time                  0
queen status          0
dtype: int64
Ukupan broj nedostajucih vrednosti:  1010

Tipovi podataka
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1275 entries, 0 to 1274
Data columns (total 23 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   device            1275 non-null   int64  
 1   hive number       1275 non-null   int64  
 2   date              1275 non-null   object 

TypeError: Could not convert ['2022-06-08 14:52:282022-06-08 15:51:412022-06-08 17:21:532022-06-08 18:20:592022-06-08 19:20:042022-06-08 20:19:132022-06-08 21:18:222022-06-08 22:17:322022-06-08 23:16:442022-06-09 0:15:562022-06-09 1:15:082022-06-09 2:14:242022-06-09 3:13:352022-06-09 4:12:492022-06-09 5:12:032022-06-09 6:11:162022-06-09 7:10:322022-06-09 8:09:592022-06-09 9:09:252022-06-09 10:08:512022-06-09 11:08:212022-06-09 12:07:532022-06-09 13:07:212022-06-09 14:06:062022-06-09 15:05:102022-06-09 16:04:232022-06-09 17:03:352022-06-09 18:49:092022-06-09 19:48:222022-06-09 20:47:282022-06-09 21:46:362022-06-09 22:45:432022-06-09 23:44:542022-06-10 0:44:042022-06-10 1:43:172022-06-10 2:42:312022-06-10 3:41:452022-06-10 4:40:572022-06-10 5:40:102022-06-10 6:39:262022-06-10 7:39:022022-06-10 8:38:242022-06-10 9:37:532022-06-10 10:37:282022-06-10 11:36:552022-06-10 12:36:082022-06-10 13:35:202022-06-10 14:34:282022-06-10 15:33:292022-06-10 16:32:452022-06-10 18:44:382022-06-10 19:43:442

## Provera i izbacivanje duplikata iz dataset-a:

In [ ]:
print(df.shape)
duplicates = df.duplicated()
print(duplicates[duplicates == True])
df = df.drop_duplicates()
print(df.shape)